In [1]:
import pandas as pd
GameInfo = pd.read_csv( 'C:/Users/Maedeh/Desktop/TenisMatchPro/data/interim3/GameInfo.csv')
MatchEventInfo = pd.read_csv( 'C:/Users/Maedeh/Desktop/TenisMatchPro/data/interim3/MatchEventInfo.csv')
MatchHomeTeamInfo = pd.read_csv( 'C:/Users/Maedeh/Desktop/TenisMatchPro/data/interim3/MatchHomeTeamInfo.csv')
MatchAwayTeamInfo = pd.read_csv( 'C:/Users/Maedeh/Desktop/TenisMatchPro/data/interim3/MatchAwayTeamInfo.csv')

In [75]:
len(MatchEventInfo['match_id'].unique())

9319

In [76]:
len(GameInfo['match_id'].unique())

6555

In [47]:
len(MatchEventInfo['match_id'].loc[MatchEventInfo['final_result_only'] == 1 ].unique())

7047

In [84]:
MatchEventInfo['match_id'].loc[MatchEventInfo['winner_code'].notnull()].unique()

array([12260075, 12260076, 12260077, ..., 12385869, 12385873, 12386383],
      dtype=int64)

nazemi

In [59]:
no=12260085
max_set = GameInfo['set_id'].loc[GameInfo['match_id'] == no].max()
max_game = GameInfo['game_id'].loc[(GameInfo['match_id'] == no) & (GameInfo['set_id'] == max_set)].max()
max_point = GameInfo['point_id'].loc[(GameInfo['match_id'] == no) & (GameInfo['set_id'] == max_set) & (GameInfo['game_id'] == max_game) ].max()
home_point = GameInfo['home_point'].loc[(GameInfo['match_id'] == no) &
                                        (GameInfo['set_id'] == max_set) & 
                                        (GameInfo['game_id'] == max_game) & 
                                        (GameInfo['point_id'] == max_point)]
away_point = GameInfo['away_point'].loc[(GameInfo['match_id'] == no) &
                                        (GameInfo['set_id'] == max_set) & 
                                        (GameInfo['game_id'] == max_game) & 
                                        (GameInfo['point_id'] == max_point)]
#home_point = 'A'
if int(home_point.item()) > int(away_point.item()):
    player_id = MatchHomeTeamInfo['player_id'].loc[MatchHomeTeamInfo['match_id'] == no]
if int(home_point.item()) < int(away_point.item()) :
    player_id = MatchAwayTeamInfo['player_id'].loc[MatchHomeTeamInfo['match_id'] == no]
player_id

7    192013
Name: player_id, dtype: int64

khodam

In [23]:
all_players = pd.concat ([MatchHomeTeamInfo , MatchAwayTeamInfo])
player = pd.DataFrame(all_players['player_id'].unique() , columns = ['player_id'])
player['no_win'] = 0
#player.loc[player['no_win'] != 0]

,player_id,no_win


In [50]:
match_ids = MatchEventInfo['match_id'].loc[MatchEventInfo['final_result_only'] == 1 ].unique()

In [28]:
match_ids = MatchEventInfo['match_id'].loc[MatchEventInfo['winner_code'].notnull()].unique()
match_ids
for no in match_ids:
    print(no)
t

12260075
12260076
12260077
12260078
12260080
12260081
12260083
12260085
12261594
12261595
12261596
12276769
12276772
12276773
12276774
12276775
12276776
12276781
12276783
12276806
12276816
12276857
12276861
12276862
12276863
12276864
12276924
12276925
12276927
12276928
12276929
12276934
12276937
12276938
12276975
12276976
12276978
12276981
12276983
12276986
12276987
12287004
12287005
12287006
12287008
12287009
12287011
12287012
12287013
12288044
12288048
12288417
12288419
12288420
12288421
12288918
12288919
12288920
12288921
12288922
12288923
12288924
12288925
12288926
12288929
12288930
12288932
12288933
12289656
12289658
12289660
12289663
12289664
12289667
12289668
12289670
12289671
12289672
12289675
12289677
12289678
12289690
12289691
12290721
12291185
12291277
12291304
12291487
12291488
12291489
12291490
12291491
12291492
12291493
12291497
12291498
12291506
12291593
12291657
12291659
12291660
12291698
12291700
12291702
12291704
12291706
12291709
12291711
12291712
12291714
12291819
1

0

In [ ]:
match_ids = MatchEventInfo['match_id'].loc[MatchEventInfo['final_result_only'] == 1 ].unique()

In [64]:
match_ids = MatchEventInfo['match_id'].loc[MatchEventInfo['final_result_only'] == 1 ].unique()
MatchEventInf = MatchEventInfo.loc[MatchEventInfo['final_result_only'] == 1 ].unique()
#match_ids = MatchEventInfo['match_id'].loc[MatchEventInfo['winner_code'].notnull()].unique()
#match_ids = [12260075, 12260076, 12260077, 12260078]
for no in match_ids:
    print(no)
    winer_code = MatchEventInfo ['winner_code'].loc[ MatchEventInfo['match_id'] == no ]
    print(winer_code)
    if int(winer_code) == 1 :
        PI = int(MatchHomeTeamInfo['player_id'].loc[MatchHomeTeamInfo['match_id'] == no])
        print (PI)
        player['no_win'].loc[player['player_id'] ==  PI] = player['no_win'].loc[player['player_id'] ==  PI] + 1
    if int(winer_code) == 2 :
        PI = int(MatchAwayTeamInfo['player_id'].loc[MatchHomeTeamInfo['match_id'] == no])
        print (PI)
        player['no_win'].loc[player['player_id'] ==  PI] = player['no_win'].loc[player['player_id'] ==  PI] + 1

12260075
0    2.0
Name: winner_code, dtype: float64
254742
12260076
1    1.0
Name: winner_code, dtype: float64
206570
12260077
2    2.0
Name: winner_code, dtype: float64
275923
12260080
4    1.0
Name: winner_code, dtype: float64
221012
12260081
5      1.0
596    1.0
Name: winner_code, dtype: float64


C:\Users\Maedeh\AppData\Local\Temp\ipykernel_15524\485105845.py:7: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  if int(winer_code) == 1 :
C:\Users\Maedeh\AppData\Local\Temp\ipykernel_15524\485105845.py:11: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  if int(winer_code) == 2 :
C:\Users\Maedeh\AppData\Local\Temp\ipykernel_15524\485105845.py:12: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  PI = int(MatchAwayTeamInfo['player_id'].loc[MatchHomeTeamInfo['match_id'] == no])
C:\Users\Maedeh\AppData\Local\Temp\ipykernel_15524\485105845.py:14: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but wh

TypeError: cannot convert the series to <class 'int'>

In [ ]:
winer_code = MatchEventInfo ['winner_code'].loc[ GameInfo['match_id'] == no ]
if int(winer_code) == 1 :
    player_id = MatchHomeTeamInfo['player_id'].loc[MatchHomeTeamInfo['match_id'] == no]
if int(winer_code) == 2 :
    player_id = MatchAwayTeamInfo['player_id'].loc[MatchHomeTeamInfo['match_id'] == no] 

In [23]:

GameInfo['point_id'].loc[(GameInfo['match_id'] == no) & (GameInfo['set_id'] == max_set)]

458557    0
458558    1
458559    2
458560    3
458561    0
         ..
487363    2
487364    3
487365    4
487366    5
487367    6
Name: point_id, Length: 90, dtype: int64